#### Controlliamo che diano lo stesso output i due checkpoint

In [1]:
import torch

ckpt_path_original = "./DeCUR/checkpoint/KAIST_Training_forSSL_pretraining_25/checkpoint_0014.pth"
ckpt_path_rewired = "./ICAFusion/checkpoint_0014_REWIRED_first636.pth"
ckpt_path_densedecur = "./DenseDeCUR/checkpoint/checkpoint_0003.pth"

original_checkpoint = torch.load(ckpt_path_original, map_location="cpu", weights_only=False)
rewired_checkpoint = torch.load(ckpt_path_rewired, map_location="cpu", weights_only=False)
densedecur_checkpoint = torch.load(ckpt_path_densedecur, map_location="cpu", weights_only=False)

print("Original checkpoint keys: {}".format(original_checkpoint.keys()))
print("Rewired checkpoint keys: {}".format(rewired_checkpoint.keys()))
print("densedecur checkpoint keys: {}".format(densedecur_checkpoint.keys()))

print("Original model keys: {}".format(original_checkpoint['model'].keys()))
print("Rewired model keys: {}".format(rewired_checkpoint['model'].keys()))
print("densedecur model keys: {}".format(densedecur_checkpoint['model'].keys()))

print("length original model keys: {}".format(len(original_checkpoint['model'].keys())))
print("length rewired model keys: {}".format(len(rewired_checkpoint['model'].keys())))
print("densedecur model keys: {}".format(len(densedecur_checkpoint['model'].keys())))

print("Gli originali è normale siano più lunghi! quelli hanno anche i parametri del proiettore, che ho tolto.")

Original checkpoint keys: dict_keys(['epoch', 'model', 'optimizer'])
Rewired checkpoint keys: dict_keys(['epoch', 'model', 'optimizer'])
densedecur checkpoint keys: dict_keys(['epoch', 'model', 'optimizer'])
Original model keys: odict_keys(['module.backbone_1.conv1.weight', 'module.backbone_1.bn1.weight', 'module.backbone_1.bn1.bias', 'module.backbone_1.bn1.running_mean', 'module.backbone_1.bn1.running_var', 'module.backbone_1.bn1.num_batches_tracked', 'module.backbone_1.layer1.0.conv1.weight', 'module.backbone_1.layer1.0.bn1.weight', 'module.backbone_1.layer1.0.bn1.bias', 'module.backbone_1.layer1.0.bn1.running_mean', 'module.backbone_1.layer1.0.bn1.running_var', 'module.backbone_1.layer1.0.bn1.num_batches_tracked', 'module.backbone_1.layer1.0.conv2.weight', 'module.backbone_1.layer1.0.bn2.weight', 'module.backbone_1.layer1.0.bn2.bias', 'module.backbone_1.layer1.0.bn2.running_mean', 'module.backbone_1.layer1.0.bn2.running_var', 'module.backbone_1.layer1.0.bn2.num_batches_tracked', 'mo

#### Carico modello ICAFusion

In [2]:
from ICAFusion.models.yolo_test import Model

config = "./ICAFusion/models/transformer/yolov5_ResNet50_Transfusion_kaist.yaml"
nc = 1        # number of classes (NON CERTO)
anchors = 0   # number of anchors (NON CERTO)

print("Anche qui mancano delle chiavi, ma è normale: abbiamo caricato solo le backbone.")
# model = Model(opt.cfg or ckpt['model'].yaml, ch=3, nc=nc, anchors=hyp.get('anchors')).to(device)
model_icafusion = Model(config, ch=3, nc=nc, anchors=anchors).to("cpu")
model_icafusion.load_state_dict(rewired_checkpoint['model'], strict=False)



/mnt/proj3/eu-25-19/davide_secco/miniconda3/envs/icafusion/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/mnt/proj3/eu-25-19/davide_secco/miniconda3/envs/icafusion/lib/python3.10/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


Anche qui mancano delle chiavi, ma è normale: abbiamo caricato solo le backbone.


_IncompatibleKeys(missing_keys=['model.10.pos_emb_vis', 'model.10.pos_emb_ir', 'model.10.vis_coefficient.w1', 'model.10.vis_coefficient.w2', 'model.10.ir_coefficient.w1', 'model.10.ir_coefficient.w2', 'model.10.crosstransformer.0.ln_input.weight', 'model.10.crosstransformer.0.ln_input.bias', 'model.10.crosstransformer.0.ln_output.weight', 'model.10.crosstransformer.0.ln_output.bias', 'model.10.crosstransformer.0.crossatt.que_proj_vis.weight', 'model.10.crosstransformer.0.crossatt.que_proj_vis.bias', 'model.10.crosstransformer.0.crossatt.key_proj_vis.weight', 'model.10.crosstransformer.0.crossatt.key_proj_vis.bias', 'model.10.crosstransformer.0.crossatt.val_proj_vis.weight', 'model.10.crosstransformer.0.crossatt.val_proj_vis.bias', 'model.10.crosstransformer.0.crossatt.que_proj_ir.weight', 'model.10.crosstransformer.0.crossatt.que_proj_ir.bias', 'model.10.crosstransformer.0.crossatt.key_proj_ir.weight', 'model.10.crosstransformer.0.crossatt.key_proj_ir.bias', 'model.10.crosstransformer.

#### Carico modello di DeCUR con i suoi pesi

In [3]:
from DeCUR.src.pretrain.models.decur import DeCUR
from argparse import Namespace

args = Namespace(backbone="resnet50", mode="rgb thermal", rda=False, projector="8192-8192-8192", batch_size=1)
model_DeCUR_original = DeCUR(args).to("cpu")

# Da questo print vedo che i nomi dei layer sono diversi (c'è un "module" in più in DeCUR)
print("Keys dei checkpoints scaricati da DeCUR: {}".format(original_checkpoint['model'].keys()))
print(f"Key che vuole DeCUR                                {list(model_DeCUR_original.state_dict().keys())}")

# prendi solo lo state_dict e togli 'module.' una sola volta in testa
original_checkpoint["model"] = {k.replace('module.', '', 1): v for k, v in original_checkpoint["model"].items()}

model_DeCUR_original.load_state_dict(original_checkpoint['model'], strict=False)


# print("DeCUR rewired model keys: {}".format(rewired_checkpoint['model'].keys()))
# model_DeCUR_rewired = DeCUR(args).to("cpu")
# model_DeCUR_rewired.load_state_dict(rewired_checkpoint['model'], strict=False)

/mnt/proj3/eu-25-19/davide_secco/miniconda3/envs/icafusion/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/mnt/proj3/eu-25-19/davide_secco/miniconda3/envs/icafusion/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Keys dei checkpoints scaricati da DeCUR: odict_keys(['module.backbone_1.conv1.weight', 'module.backbone_1.bn1.weight', 'module.backbone_1.bn1.bias', 'module.backbone_1.bn1.running_mean', 'module.backbone_1.bn1.running_var', 'module.backbone_1.bn1.num_batches_tracked', 'module.backbone_1.layer1.0.conv1.weight', 'module.backbone_1.layer1.0.bn1.weight', 'module.backbone_1.layer1.0.bn1.bias', 'module.backbone_1.layer1.0.bn1.running_mean', 'module.backbone_1.layer1.0.bn1.running_var', 'module.backbone_1.layer1.0.bn1.num_batches_tracked', 'module.backbone_1.layer1.0.conv2.weight', 'module.backbone_1.layer1.0.bn2.weight', 'module.backbone_1.layer1.0.bn2.bias', 'module.backbone_1.layer1.0.bn2.running_mean', 'module.backbone_1.layer1.0.bn2.running_var', 'module.backbone_1.layer1.0.bn2.num_batches_tracked', 'module.backbone_1.layer1.0.conv3.weight', 'module.backbone_1.layer1.0.bn3.weight', 'module.backbone_1.layer1.0.bn3.bias', 'module.backbone_1.layer1.0.bn3.running_mean', 'module.backbone_1.la

<All keys matched successfully>

In [4]:
import re
import torch
from typing import Iterable, Optional, Sequence, Tuple, List

def count_params(m: torch.nn.Module) -> int:
    return sum(p.numel() for p in m.parameters() if p.requires_grad)

def print_named_modules_flat(model: torch.nn.Module, max_rows: Optional[int] = None):
    rows = [(name, m.__class__.__name__, count_params(m)) for name, m in model.named_modules()]
    if max_rows is not None:
        rows = rows[:max_rows]
    for n, cls, npar in rows:
        print(f"{n:50s} -> {cls:25s} params={npar}")

# ---------- helper di auto-detection ----------

def _sd_prefixes(sd_keys: Sequence[str], pat: str) -> List[str]:
    """Estrae i prefissi univoci prima del primo '.' che matchano pat."""
    pref = []
    for k in sd_keys:
        if re.match(pat, k):
            p = k.split('.', 1)[0]
            if p not in pref:
                pref.append(p)
    return pref

def _detect_decur_prefixes(model: torch.nn.Module, include_extras: bool = True) -> List[str]:
    sd_keys = list(model.state_dict().keys())
    bbs = _sd_prefixes(sd_keys, r'^backbone_\d+\.')  # es. ['backbone_1','backbone_2']
    if not bbs:
        return []
    if include_extras:
        for extra in ['projector1', 'projector2', 'bn']:
            if any(k.startswith(extra + '.') for k in sd_keys):
                bbs.append(extra)
    return bbs

def _collect_icafusion_indices(model: torch.nn.Module) -> List[int]:
    """Raccoglie gli indici n per moduli top-level chiamati 'model.n'."""
    idxs = []
    for name, m in model.named_modules():
        mobj = re.fullmatch(r'model\.(\d+)', name)
        if mobj:
            idxs.append(int(mobj.group(1)))
    return sorted(set(idxs))

def _infer_icafusion_cutoff(model: torch.nn.Module) -> int:
    """
    Prova a trovare l'indice minimo 't' tale che per qualche sotto-modulo 'model.t.*'
    la classe contiene 'Transformer' o 'Fusion'. Restituisce t, così i backbone sono < t.
    Se non trova nulla, restituisce max_indice+1 (cioè tutti).
    """
    candidates = []
    for name, m in model.named_modules():
        mobj = re.match(r'model\.(\d+)(?:\.|$)', name)
        if mobj:
            i = int(mobj.group(1))
            cls = m.__class__.__name__.lower()
            if ('transformer' in cls) or ('fusion' in cls):
                candidates.append(i)
    if candidates:
        return min(candidates)
    idxs = _collect_icafusion_indices(model)
    return (max(idxs) + 1) if idxs else 0

def _icafusion_prefixes(model: torch.nn.Module,
                        streams: Optional[Iterable[Tuple[int, int]]] = None) -> List[str]:
    if streams:
        indices = []
        for a, b in streams:
            indices.extend(range(a, b + 1))
        indices = sorted(set(indices))
    else:
        cutoff = _infer_icafusion_cutoff(model)
        indices = [i for i in _collect_icafusion_indices(model) if i < cutoff]
    return [f'model.{i}' for i in indices]

def _auto_prefixes(model: torch.nn.Module,
                   streams: Optional[Iterable[Tuple[int, int]]] = None,
                   include_extras: bool = True) -> List[str]:
    # 1) prova schema DeCUR
    decur = _detect_decur_prefixes(model, include_extras=include_extras)
    if decur:
        return decur
    # 2) fallback a schema ICAFusion (con streams opzionale)
    icaf = _icafusion_prefixes(model, streams=streams)
    return icaf

# ---------- API principali ----------

def print_backbone_named_modules(model: torch.nn.Module,
                                 streams: Optional[Iterable[Tuple[int, int]]] = None,
                                 include_extras: bool = False,
                                 max_rows: Optional[int] = None):
    """
    Stampa i moduli appartenenti ai prefissi auto-rilevati.
    - DeCUR: 'backbone_1','backbone_2' (+ extras se include_extras=True)
    - ICAFusion: 'model.i' per gli i ricavati da streams o inferiti
    """
    prefixes = _auto_prefixes(model, streams=streams, include_extras=include_extras)
    rows = []
    for name, m in model.named_modules():
        if any(name == p or name.startswith(p + ".") for p in prefixes):
            rows.append((name, m.__class__.__name__, count_params(m)))
    if max_rows is not None:
        rows = rows[:max_rows]
    for n, cls, npar in rows:
        print(f"{n:50s} -> {cls:25s} params={npar}")

def print_module_to_state_dict_keys(model: torch.nn.Module,
                                    filter_prefixes: Optional[Iterable[str]] = None,
                                    streams: Optional[Iterable[Tuple[int, int]]] = None,
                                    include_extras: bool = True):
    """
    Mappa modulo -> chiavi dello state_dict.
    - Se filter_prefixes è None, autodetect (DeCUR o ICAFusion).
    - Per ICAFusion puoi passare streams per riprodurre esattamente il tuo slicing.
    """
    sd = model.state_dict()
    sd_keys = list(sd.keys())

    def keys_for_module(name: str):
        if not name:
            return []
        prefix = name + "."
        return [k for k in sd_keys if k.startswith(prefix)]

    if filter_prefixes is None:
        filter_prefixes = _auto_prefixes(model, streams=streams, include_extras=include_extras)

    for name, m in model.named_modules():
        if filter_prefixes and not any(name == p or name.startswith(p + ".") for p in filter_prefixes):
            continue
        ks = keys_for_module(name)
        if ks:
            print(f"[{name}] ({m.__class__.__name__})")
            for k in ks:
                t = sd[k]
                try:
                    shp = tuple(t.shape)
                except Exception:
                    shp = ()
                print(f"   - {k:70s}  shape={shp}")
            print()


### Visualizzaiamo le strutture delle reti per i nomi

#### Decur

In [5]:
print_backbone_named_modules(model_DeCUR_original, include_extras=True, max_rows=40)
print_module_to_state_dict_keys(model_DeCUR_original, filter_prefixes=None, include_extras=True)


backbone_1                                         -> ResNet                    params=23508032
backbone_1.conv1                                   -> Conv2d                    params=9408
backbone_1.bn1                                     -> BatchNorm2d               params=128
backbone_1.relu                                    -> ReLU                      params=0
backbone_1.maxpool                                 -> MaxPool2d                 params=0
backbone_1.layer1                                  -> Sequential                params=215808
backbone_1.layer1.0                                -> Bottleneck                params=75008
backbone_1.layer1.0.conv1                          -> Conv2d                    params=4096
backbone_1.layer1.0.bn1                            -> BatchNorm2d               params=128
backbone_1.layer1.0.conv2                          -> Conv2d                    params=36864
backbone_1.layer1.0.bn2                            -> BatchNorm2d               

#### ICAFusion

In [6]:
# Mantieni i tuoi range
streams = ((0,4), (5,9)) 
print_backbone_named_modules(model_icafusion, streams=streams, max_rows=60)
print_module_to_state_dict_keys(model_icafusion, streams=streams, include_extras=False)


model.0                                            -> ResNetlayer               params=9536
model.0.layer                                      -> Sequential                params=9536
model.0.layer.0                                    -> Conv2d                    params=9408
model.0.layer.1                                    -> BatchNorm2d               params=128
model.0.layer.2                                    -> ReLU                      params=0
model.0.layer.3                                    -> MaxPool2d                 params=0
model.1                                            -> ResNetlayer               params=215808
model.1.layer                                      -> Sequential                params=215808
model.1.layer.0                                    -> ResNetblock               params=75008
model.1.layer.0.conv1                              -> Conv2d                    params=4096
model.1.layer.0.bn1                                -> BatchNorm2d               pa

In [7]:
from PIL import Image
import torch
import torchvision.transforms as T

def _tfm(size, in_channels):
    ops = [T.Resize(size), T.ToTensor()]
    # normalizzazione: se i backbone hanno pesi ImageNet, usa mean/std ImageNet
    if in_channels == 3:
        ops.append(T.Normalize(mean=[0.485, 0.456, 0.406],
                               std=[0.229, 0.224, 0.225]))
    elif in_channels == 1:
        # semplice z-score centrato (puoi cambiare a tuo piacimento)
        ops.append(T.Normalize(mean=[0.5], std=[0.5]))
    return T.Compose(ops)

def load_image_for_backbone(path: str, in_channels: int, size=(640, 640), device="cpu"):
    img = Image.open(path)
    # adatta il numero di canali alla conv1 attesa
    if in_channels == 3:
        img = img.convert("RGB")      # se era L/greyscale, replica su 3 canali
    elif in_channels == 1:
        img = img.convert("L")        # se era RGB, riduce a 1 canale
    else:
        # fallback semplice: porta a RGB e poi ritaglia i primi in_channels o ripete
        img = img.convert("RGB")
    x = _tfm(size, min(in_channels, 3))(img)  # [C,H,W] normalizzato
    if in_channels not in (1,3):
        # porta a in_channels (es. 2, 4, …) ripetendo/tagliando
        if x.shape[0] < in_channels:
            rep = (in_channels + x.shape[0] - 1) // x.shape[0]
            x = x.repeat(rep, 1, 1)[:in_channels]
        else:
            x = x[:in_channels]
    x = x.unsqueeze(0).to(device)      # [1,C,H,W]
    return x

def make_decur_inputs_from_paths(model, path_rgb, path_ir, size=(640,640), device="cpu"):
    sd = model.state_dict()
    C1 = sd["backbone_1.conv1.weight"].shape[1]
    C2 = sd["backbone_2.conv1.weight"].shape[1]
    x1 = load_image_for_backbone(path_rgb, C1, size=size, device=device)
    x2 = load_image_for_backbone(path_ir,  C2, size=size, device=device)
    # nessuna augment: usa la stessa immagine per le due viste
    return x1, x1, x2, x2


In [18]:
import re
import torch
from PIL import Image
import torchvision.transforms as T
from collections import OrderedDict
from typing import Iterable, Optional, Tuple

# -------------------- helpers immagine --------------------
def _build_transform(size_hw: Tuple[int,int], in_channels: int, normalize: bool):
    H, W = size_hw
    ops = [T.Resize((H, W)), T.ToTensor()]
    if normalize:
        if in_channels == 3:
            ops.append(T.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]))
        elif in_channels == 1:
            ops.append(T.Normalize(mean=[0.5], std=[0.5]))
    return T.Compose(ops)

def _to_channels(x: torch.Tensor, c: int) -> torch.Tensor:
    """Adatta x [B,C,H,W] a C=c ripetendo/tagliando."""
    B,C,H,W = x.shape
    if C == c:
        return x
    if C > c:
        return x[:, :c]
    rep = (c + C - 1) // C
    return x.repeat(1, rep, 1, 1)[:, :c]

def _load_image_as_tensor(path: str, in_channels: int, size_hw: Tuple[int,int],
                          device: torch.device, normalize: bool):
    img = Image.open(path)
    img = img.convert("RGB") if in_channels != 1 else img.convert("L")
    x = _build_transform(size_hw, 3 if in_channels != 1 else 1, normalize)(img)  # [C,H,W]
    x = x.unsqueeze(0).to(device)  # [1,C,H,W]
    return _to_channels(x, in_channels)

# -------------------- helpers modello --------------------
def _is_decur(model: torch.nn.Module) -> bool:
    sd = model.state_dict().keys()
    return any(k.startswith("backbone_1.") for k in sd) or hasattr(model, "backbone_1")

def _is_icafusion(model: torch.nn.Module) -> bool:
    sd = model.state_dict().keys()
    return any(re.match(r"^model\.\d+\.", k) for k in sd)

# -------------------- funzione principale --------------------
def get_backbone_features(
    model: torch.nn.Module,
    TARGET_PREFIXES: Iterable[str],
    path_vis: Optional[str] = None,
    path_ir:  Optional[str] = None,
    size_hw: Tuple[int,int] = (640,640),
    normalize: bool = True,
    channels_hint: Optional[Tuple[Optional[int], Optional[int]]] = None,  # (C_vis, C_ir)
):
    """
    - DeCUR: esegue solo backbone_1/backbone_2 in base ai TARGET_PREFIXES.
    - ICAFusion: chiama model(x_vis, x_ir). Se c'è mismatch canali, adatta automaticamente.
    - channels_hint: forza (C_vis, C_ir). Es: (3,3) per replicare IR su 3 canali.
    """
    model.eval()
    device = next(model.parameters()).device

    feat_maps = OrderedDict()
    handles = []

    def make_hook(name):
        def hook(module, inp, out):
            y = out[0] if isinstance(out, (tuple, list)) else out
            if torch.is_tensor(y):
                feat_maps[name] = y.detach().cpu()
        return hook

    for name, module in model.named_modules():
        if any(name == p or name.startswith(p + ".") for p in TARGET_PREFIXES):
            print(f"name: {name}")
            handles.append(module.register_forward_hook(make_hook(name)))

    H, W = size_hw
    sd = model.state_dict()

    try:
        if _is_decur(model):
            # ----- DECUR -----
            C1 = sd["backbone_1.conv1.weight"].shape[1]
            C2 = sd["backbone_2.conv1.weight"].shape[1]
            x1 = _load_image_as_tensor(path_vis, C1, size_hw, device, normalize) if path_vis else torch.zeros(1, C1, H, W, device=device)
            x2 = _load_image_as_tensor(path_ir,  C2, size_hw, device, normalize) if path_ir  else torch.zeros(1, C2, H, W, device=device)
            need_b1 = any(p.startswith("backbone_1") for p in TARGET_PREFIXES)
            need_b2 = any(p.startswith("backbone_2") for p in TARGET_PREFIXES)
            with torch.no_grad():
                if need_b1: _ = model.backbone_1(x1)
                if need_b2: _ = model.backbone_2(x2)

        elif _is_icafusion(model):
            # ----- ICAFUSION -----
            # canali predefiniti: prova a prendere dall'hint, altrimenti (3,1) come default,
            # ma correggiamo automaticamente se il forward alza un errore di canali.
            C_vis = channels_hint[0] if (channels_hint and channels_hint[0]) else 3
            C_ir  = channels_hint[1] if (channels_hint and channels_hint[1]) else 1

            x_vis = _load_image_as_tensor(path_vis, C_vis, size_hw, device, normalize) if path_vis else torch.zeros(1, C_vis, H, W, device=device)
            x_ir  = _load_image_as_tensor(path_ir,  C_ir,  size_hw, device, normalize) if path_ir  else torch.zeros(1, C_ir,  H, W, device=device)

            def _try_forward(xv, xi):
                with torch.no_grad():
                    return model(xv, xi)

            try:
                _ = _try_forward(x_vis, x_ir)
            except RuntimeError as e:
                msg = str(e)
                # prova ad estrarre expected/got e adattare
                m = re.search(r"expected input\[\d+,\s*(\d+),.*\] to have (\d+) channels, but got (\d+) channels", msg)
                if m:
                    got_c = int(m.group(1)) if False else None  # non sempre affidabile
                # fallback: cerca "expected .* to have X channels, but got Y"
                m2 = re.search(r"to have (\d+) channels, but got (\d+) channels", msg)
                if m2:
                    exp_c, got_c = int(m2.group(1)), int(m2.group(2))
                    # se l'errore viene dal ramo IR, spesso basta portare IR a exp_c
                    x_ir = _to_channels(x_ir, exp_c)
                    _ = _try_forward(x_vis, x_ir)
                else:
                    # se non parsabile, prova prima IR->3, poi IR->1
                    try:
                        _ = _try_forward(x_vis, _to_channels(x_ir, 3))
                    except RuntimeError:
                        _ = _try_forward(x_vis, _to_channels(x_ir, 1))

        else:
            # ----- SCONOSCIUTO: una sola immagine -----
            C = 3
            x = _load_image_as_tensor(path_vis, C, size_hw, device, normalize) if path_vis else torch.zeros(1, C, H, W, device=device)
            with torch.no_grad():
                try:
                    _ = model(x)
                except TypeError:
                    _ = model(x, x)

    finally:
        for h in handles:
            h.remove()

    # report
    print("=== BACKBONE FEATURES ===")
    for k, v in feat_maps.items():
        m = v.float().mean().item() if v.numel() else float("nan")
        s = v.float().std().item()  if v.numel() else float("nan")
        print(f"{k:45s} -> shape={tuple(v.shape)}  mean={m:.6f}  std={s:.6f}")

    torch.save({k: v.contiguous() for k, v in feat_maps.items()}, "backbone_features.pt")
    print("Salvate feature in: backbone_features.pt")
    return feat_maps


In [24]:
get_backbone_features(
    model_DeCUR_original,
    TARGET_PREFIXES=["backbone_2.layer4.2.conv3"],
    path_vis="/scratch/project/eu-25-19/kaist-cvpr15/visible/train/set03_V000_I00000.jpg",
    path_ir ="/scratch/project/eu-25-19/kaist-cvpr15/infrared/train/set03_V000_I00000.jpg",
    size_hw=(640,512)
)


name: backbone_2.layer4.2.conv3
=== BACKBONE FEATURES ===
backbone_2.layer4.2.conv3                     -> shape=(1, 2048, 20, 16)  mean=-3.965909  std=4.707794
Salvate feature in: backbone_features.pt


OrderedDict([('backbone_2.layer4.2.conv3',
              tensor([[[[-4.34693e+00, -6.18248e+00, -5.91949e+00,  ..., -4.03908e+00, -4.07903e+00, -2.44527e+00],
                        [-5.36923e+00, -8.53021e+00, -8.55282e+00,  ..., -4.98833e+00, -5.20419e+00, -3.26572e+00],
                        [-4.88702e+00, -8.13502e+00, -8.47552e+00,  ..., -4.27639e+00, -4.70956e+00, -2.90724e+00],
                        ...,
                        [-5.29322e+00, -8.29219e+00, -7.60135e+00,  ..., -7.38597e+00, -8.85375e+00, -5.56328e+00],
                        [-4.40623e+00, -6.82739e+00, -6.29319e+00,  ..., -6.03683e+00, -7.08689e+00, -4.13568e+00],
                        [-1.91058e+00, -3.25070e+00, -2.97180e+00,  ..., -2.82715e+00, -3.33221e+00, -1.86174e+00]],
              
                       [[-6.08355e+00, -9.72746e+00, -9.54466e+00,  ..., -6.41394e+00, -6.31168e+00, -3.67109e+00],
                        [-9.90825e+00, -1.54736e+01, -1.53561e+01,  ..., -9.50199e+00, -9.83318e+00,

In [23]:
get_backbone_features(
    model_icafusion,
    TARGET_PREFIXES=["model.9.layer.2.conv3"],
    path_vis="/scratch/project/eu-25-19/kaist-cvpr15/visible/train/set03_V000_I00000.jpg",
    path_ir ="/scratch/project/eu-25-19/kaist-cvpr15/infrared/train/set03_V000_I00000.jpg",
    size_hw=(640,512)
)


name: model.9.layer.2.conv3
=== BACKBONE FEATURES ===
model.9.layer.2.conv3                         -> shape=(1, 2048, 20, 16)  mean=-0.019353  std=0.059369
Salvate feature in: backbone_features.pt


OrderedDict([('model.9.layer.2.conv3',
              tensor([[[[-2.49873e-02, -1.49198e-02, -1.15756e-02,  ..., -1.94884e-02, -1.92943e-02, -2.52257e-02],
                        [-2.12052e-02,  8.64556e-03,  1.16320e-02,  ..., -1.38485e-02, -2.23635e-03, -8.35405e-03],
                        [-1.32984e-02,  1.78562e-02,  2.51404e-02,  ..., -2.49769e-02, -7.62268e-03, -7.36351e-03],
                        ...,
                        [-1.19360e-02,  6.08391e-02,  9.14337e-02,  ...,  6.86781e-02,  6.33743e-02, -5.69978e-03],
                        [-4.85356e-02, -1.06613e-02,  9.12999e-03,  ..., -4.81435e-03, -6.37071e-03, -4.81652e-02],
                        [-6.98366e-02, -6.64856e-02, -6.08367e-02,  ..., -6.09931e-02, -6.27750e-02, -6.92670e-02]],
              
                       [[-1.55102e-03, -5.06380e-03, -5.56248e-03,  ..., -1.06237e-02, -4.36899e-03, -1.36982e-03],
                        [ 1.89938e-02,  2.29976e-02,  2.11606e-02,  ..., -2.13610e-02,  2.05889e-03,  8.